# Part 1: Foundations
## Chapter 1.3: Python Fundamentals: Data Types, Operators, and Memory Model

### Table of Contents
1. [Introduction: Why Low-Level Details Matter](#Introduction:-Why-Low-Level-Details-Matter-for-High-Level-Research)
2. [Numerical Representation and Stability](#1.-Numerical-Representation-and-Stability)
    - [Integer Representation](#1.1-Integer-Representation)
    - [Floating-Point Representation: IEEE 754](#1.2-Floating-Point-Representation:-The-IEEE-754-Standard)
    - [Special Floating-Point Values: `inf` and `NaN`](#1.3-Special-Floating-Point-Values:-inf-and-NaN)
    - [Numerical Stability: Catastrophic Cancellation](#1.4-Numerical-Stability:-Catastrophic-Cancellation)
    - [The `Decimal` Type](#1.5-The-Decimal-Type:-For-When-Base-10-Precision-Matters)
3. [Core Data Types](#2.-Core-Data-Types)
    - [Booleans (`bool`)](#2.1-Booleans-(bool))
    - [The `None` Singleton](#2.2-The-None-Singleton)
    - [Strings (`str`)](#2.3-Strings-(str))
4. [Variables, Objects, and Mutability](#3.-Variables,-Objects,-and-Mutability)
    - [The `is` Operator vs. `==`](#3.1-The-is-Operator-vs.-==)
    - [Shallow vs. Deep Copying](#3.2-Shallow-vs.-Deep-Copying)
    - [Garbage Collection](#3.3-Garbage-Collection:-How-Python-Reclaims-Memory)
    - [Mutability and Function Arguments: A Common Pitfall](#3.4-Mutability-and-Function-Arguments:-A-Common-Pitfall)
5. [Operators](#4.-Operators)
    - [Operator Precedence](#4.1-Arithmetic-and-Comparison-Operators)
    - [Bitwise Operators](#4.2-Bitwise-Operators)
6. [The Python Data Model](#5.-The-Python-Data-Model:-Understanding-"Under-the-Hood")
7. [Advanced Type Hinting](#6.-Advanced-Type-Hinting)
8. [Exception Handling: Writing Robust Code](#7.-Exception-Handling:-Writing-Robust-Code)
9. [Exercises](#8.-Exercises)
10. [Solutions to Exercises](#9.-Solutions-to-Exercises)

### Introduction: Why Low-Level Details Matter for High-Level Research

It can be tempting to view the low-level details of a programming language—how it represents numbers or manages memory—as technical minutiae, far removed from the high-level intellectual work of economic modeling. This is a dangerous misconception. For a computational economist, a solid grasp of these fundamentals is not optional; it is a prerequisite for producing reliable and credible research.

Consider a few examples:
- **Simulating a Dynamic Model:** When you simulate a model over thousands of time steps, tiny floating-point representation errors can accumulate, potentially leading to a solution that is a numerical artifact rather than a true feature of the model. Understanding the limits of `float64` precision is essential for diagnosing such issues.
- **Estimating a Structural Model:** When passing large datasets or complex parameter objects to estimation functions, understanding the difference between mutable and immutable types (and shallow vs. deep copies) can be the key to debugging why a change in one part of your code is having unexpected side effects in another.
- **Writing Efficient Code:** Knowing that string concatenation in a loop creates a new object with every iteration, while appending to a list modifies it in-place, can be the difference between code that runs in seconds and code that takes hours.

This chapter lays the foundation. It introduces the core data types that Python uses to represent information, the operators used to manipulate them, and, most critically, the underlying memory model that governs how variables and data structures behave. Mastering these concepts is the first step toward writing the clear, efficient, and robust code that modern economic research demands.

### 1. Numerical Representation and Stability

Understanding how a computer represents numbers is not merely an academic exercise; it has profound practical implications for numerical stability, precision, and efficiency in computational economics. Computers store all information, including numbers, in binary format (bits, i.e., 0s and 1s).

#### 1.1 Integer Representation

Integers (`int`) represent whole numbers. In many programming languages (like C++ or Java), integers have a fixed size (e.g., 32-bit or 64-bit), which limits the range of values they can store. Python is different.

**Arbitrary-Precision Integers:** Python's integers have **unlimited precision**. A Python `int` is an object that can grow to accommodate any number of bits, limited only by the available RAM of your machine. This is a powerful convenience, as it means you never have to worry about an integer calculation overflowing (exceeding the maximum representable value), a common and sometimes subtle bug in scientific code written in lower-level languages like Fortran or C.

In [ ]:
large_int = 10**100
print(f"A googol: {large_int}")
print(f"Number of bits to represent this integer: {large_int.bit_length()}")

#### 1.2 Floating-Point Representation: The IEEE 754 Standard

Real numbers are represented using floating-point numbers (`float`). Python, like most modern languages and hardware, uses the **IEEE 754 standard** for floating-point arithmetic. Understanding this standard is crucial for diagnosing numerical issues.

A floating-point number is stored in a fixed number of bits and is composed of three parts:

| Part | Double Precision (`float64`) | Single Precision (`float32`) | Purpose |
| :--- | :--- | :--- | :--- |
| **Sign** | 1 bit | 1 bit | Determines if the number is positive (0) or negative (1). |
| **Exponent** | 11 bits | 8 bits | Determines the magnitude (scale) of the number. |
| **Mantissa** | 52 bits | 23 bits | Represents the significant digits of the number (its precision). |

**The Consequence: Finite Precision and Representation Error**
Because only a finite number of bits are used, most real numbers cannot be represented *exactly*. Numbers that are simple in base 10, like 0.1, are repeating fractions in base 2. This forces an approximation, leading to small but important **representation errors**.

##### The Precision vs. Performance Trade-off: `float64` vs. `float32`
By default, Python's `float` type is a **double-precision** number, which corresponds to `numpy.float64`. This is the standard for most scientific work, offering a high degree of precision.

However, for very large-scale simulations or machine learning models, the memory and computational overhead of using `float64` for every number can be substantial. In these cases, using **single-precision** floats (`numpy.float32`) can be a powerful optimization. A `float32` number uses half the memory of a `float64`, which can dramatically reduce the memory footprint of large arrays and potentially speed up computations, especially on hardware like GPUs that are optimized for `float32` operations. The trade-off, of course, is a reduction in precision, which makes `float32` unsuitable for problems that are highly sensitive to numerical errors (i.e., that are "ill-conditioned").

In [ ]:
# Demonstrate the memory difference between float64 and float32
import numpy as np

# Create a large array (10 million numbers)
large_array_64 = np.ones(10_000_000, dtype=np.float64)
large_array_32 = np.ones(10_000_000, dtype=np.float32)

# Get the size in megabytes (1 byte = 8 bits, 1 megabyte = 1024*1024 bytes)
mem_64 = large_array_64.nbytes / (1024 * 1024)
mem_32 = large_array_32.nbytes / (1024 * 1024)

print(f"Memory usage of float64 array: {mem_64:.2f} MB")
print(f"Memory usage of float32 array: {mem_32:.2f} MB")
print(f"Using float32 saved {mem_64 - mem_32:.2f} MB of memory.")

In [ ]:
# The classic example of floating-point inaccuracy
val1 = 0.1 + 0.2
print(f"0.1 + 0.2 = {val1:.17f}")
print(f"Is this equal to 0.3? {val1 == 0.3}\n")

# Another example
val2 = 0.1 + 0.1 + 0.1
print(f"0.1 + 0.1 + 0.1 = {val2:.17f}") # Show more precision
print(f"Is this equal to 0.3? {val2 == 0.3}\n")

# This is why you should NEVER test for exact equality with floats.
# The correct approach is to test for approximate equality within a tolerance.
import numpy as np
print(f"Are they close? {np.isclose(val1, 0.3)}")

#### 1.3 Special Floating-Point Values: `inf` and `NaN`
The IEEE 754 standard defines several special values that are critical for robust numerical programming:
- **`inf` (Infinity):** Represents positive infinity. It arises from operations like `1 / 0` or numbers exceeding the maximum representable float.
- **`-inf` (Negative Infinity):** Represents negative infinity.
- **`NaN` (Not a Number):** Represents an undefined or unrepresentable value, such as the result of `0 / 0` or `inf - inf`. `NaN` has the unique property that it is not equal to anything, including itself (`NaN != NaN`).

These values are essential because they allow computations to continue where they might otherwise crash. `numpy` provides functions to check for them: `np.isinf()`, `np.isneginf()`, `np.isposinf()`, and `np.isnan()`.

In [ ]:
a = np.array([1, 2, np.inf, -np.inf, np.nan, 0])
print(f"Original array: {a}")
print(f"Is finite: {np.isfinite(a)}")
print(f"Is infinite: {np.isinf(a)}")
print(f"Is NaN: {np.isnan(a)}")

# NaN is 'contagious' in arithmetic operations
print(f"\n1 + NaN = {1 + np.nan}")
print(f"inf * 0 = {np.inf * 0}")

#### 1.4 Numerical Stability: Catastrophic Cancellation

A more insidious problem than representation error is **catastrophic cancellation**. This occurs when you subtract two nearly equal numbers. In doing so, you can lose a significant number of correct leading digits, leaving you with a result dominated by the noise from the trailing, less-precise digits. This can be a major source of error in iterative algorithms.

**Example:** Consider the quadratic formula for $ax^2 + bx + c = 0$. One root is $x_1 = \frac{-b + \sqrt{b^2 - 4ac}}{2a}$. If $b$ is large and positive, and $4ac$ is small, then $\sqrt{b^2 - 4ac} \approx b$. The numerator becomes a subtraction of two nearly equal numbers, leading to a loss of precision. The solution is to use an alternative, mathematically equivalent formula for that root, such as the one based on Vieta's formulas: $x_1 = \frac{2c}{-b - \sqrt{b^2 - 4ac}}$.

In [ ]:
import numpy as np

# Let's solve x^2 + 10^8*x + 1 = 0
a, c = 1.0, 1.0
b = 1e8

# Use single-precision floats (float32) to make the error more obvious
a, b, c = np.float32(a), np.float32(b), np.float32(c)

# Naive formula: suffers from catastrophic cancellation
sqrt_discriminant = np.sqrt(b*b - 4*a*c)
x1_naive = (-b + sqrt_discriminant) / (2*a)

# Stable formula
x1_stable = (2*c) / (-b - sqrt_discriminant)

# The true answer is very close to -1e-8
print(f"Naive result:    {x1_naive}")
print(f"Stable result:   {x1_stable}")
print(f"True result (approx): {-1e-8}")

#### 1.5 The `Decimal` Type: For When Base-10 Precision Matters

For applications where base-10 representation errors are unacceptable, such as financial calculations or accounting, Python's `decimal` module provides the `Decimal` type. It stores numbers as base-10 representations with user-specified precision, avoiding the binary representation issues of floats.

**Rule of Thumb:** Use `float` for scientific and engineering calculations where high performance is key and small binary representation errors are acceptable. Use `Decimal` for financial and monetary calculations where the precision must match human-centric base-10 arithmetic.

In [ ]:
from decimal import Decimal, getcontext

# Set the precision (number of significant digits)
getcontext().prec = 50

# Important: Always initialize Decimals from strings to avoid introducing float errors
a = Decimal('0.1')
b = Decimal('0.1')
c = Decimal('0.1')
d = Decimal('0.3')

print(f"Using Decimals: {a + b + c}")
print(f"Is this equal to 0.3? {a + b + c == d}")

### 2. Core Data Types

#### 2.1 Booleans (`bool`)
Booleans represent the logical values `True` and `False`. They are the result of comparison and logical operations. Internally, `bool` is a subclass of `int`, where `True` has a value of 1 and `False` has a value of 0. This allows them to be used in arithmetic operations, which can be a useful trick for creating conditional switches in numerical code.

In [ ]:
is_greater = 5 > 3
print(f"5 > 3 is: {is_greater}")

# Using booleans in arithmetic as a conditional switch
price = 100
is_taxable = True
tax = 0.05 * price * is_taxable # Computes tax only if is_taxable is True (1)
print(f"Calculated tax: {tax}")

#### 2.2 The `None` Singleton
`None` is a special constant that represents the absence of a value or a null value. It is an object of its own type, `NoneType`. There is only ever one `None` object in memory during a Python session (it is a **singleton**), so the identity operator `is` is the conventional way to check for it.

It is frequently used as a sentinel value for default arguments in functions to distinguish between a user providing no input and a user providing a meaningful "falsy" value like `0`, `False`, or an empty list.

#### 2.3 Strings (`str`)
Strings are **immutable** sequences of characters used to represent text. Their immutability means that any operation that seems to modify a string actually creates a new one in memory. Python provides a rich set of methods for common string manipulations, which are essential for data cleaning.

**Formatted String Literals (f-strings):** Introduced in Python 3.6, f-strings are the modern, standard way to embed expressions inside string literals. They are readable, concise, and fast.

**Common Data Cleaning Methods:**
- `.strip()`: Removes leading and trailing whitespace.
- `.lower()`/`.upper()`: Converts the string to lowercase or uppercase.
- `.replace(old, new)`: Replaces all occurrences of a substring with another.
- `.split(delimiter)`: Splits the string into a list of substrings based on a delimiter.

In [ ]:
# F-string example
gdp_growth = 0.025
year = 2023
report = f"The GDP growth for {year} was {gdp_growth:.2%}."
print(f"F-string example: {report}")

# Data cleaning examples
raw_data_entry = "  USA, 2.5%, 2023   "

# Chaining methods together
clean_components = [s.strip() for s in raw_data_entry.strip().lower().replace('%', '').split(',')]
print(f"Chained methods: {clean_components}")

### 3. Variables, Objects, and Mutability

This is one of the most critical concepts for avoiding common pitfalls in Python.

In Python, a variable is not a container for a value, but rather a **name** or **label** that refers to an object in memory. The `id()` function returns the unique memory address of an object. This helps illustrate how Python manages variables.

- **Immutable Types:** An object whose state cannot be modified after it is created. Numbers (`int`, `float`, `Decimal`), booleans (`bool`), strings (`str`), and tuples (`tuple`) are immutable. Any operation that seems to modify an immutable object actually creates a *new object* in memory and reassigns the variable name to point to it.
- **Mutable Types:** An object whose state can be modified after it is created. Lists (`list`), dictionaries (`dict`), and sets (`set`) are mutable. Operations that modify these objects do so *in-place*, without creating a new object.

The diagrams below illustrate this critical distinction.

**Immutable Example:** When we execute `x = x + 1`, the name `x` is simply moved to point to a new integer object. The original object `42` is unchanged (and will be garbage collected if no other names point to it).

![Behavior of immutable types](../images/png/1.3-immutable-before.png)
![Behavior of immutable types](../images/png/1.3-immutable-after.png)

**Mutable Example:** When we execute `my_list.append(4)`, the list object itself is modified *in-place*. The name `my_list` continues to point to the same object, which now has a different internal state.

![Behavior of mutable types](../images/png/1.3-mutable-before.png)
![Behavior of mutable types](../images/png/1.3-mutable-after.png)

#### 3.1 The `is` Operator vs. `==`
- `==`: Checks if the *values* of two variables are equal. This is an application of the data model; `a == b` is equivalent to `a.__eq__(b)`.
- `is`: Checks if two variables point to the *exact same object* in memory (i.e., if `id(a) == id(b)`).

For small integers and strings, Python often uses a cache (an optimization called "interning"), so `is` might return `True` unexpectedly. You should almost always use `==` for equality checks, reserving `is` for checking identity, especially for singletons like `None` (e.g., `if my_var is None:`).

In [ ]:
a = [1, 2, 3]
b = [1, 2, 3]
c = a

print(f"a == b: {a == b}") # True, values are the same
print(f"a is b: {a is b}") # False, they are different objects in memory
print(f"a is c: {a is c}") # True, c is just another name for the object a refers to

#### 3.2 Shallow vs. Deep Copying
The distinction between mutable and immutable types is most critical when copying objects. Creating a copy of a mutable object that contains other mutable objects requires care.

- **Assignment (`b = a`):** This never copies the object. It only creates a new name (`b`) that points to the exact same object as `a`.
- **Shallow Copy (`b = a.copy()` or `b = copy.copy(a)`):** This creates a new top-level object, but populates it with *references* to the objects contained in the original. If the contained objects are mutable, both the original and the copy will share them.
- **Deep Copy (`b = copy.deepcopy(a)`):** This creates a new top-level object and then *recursively* creates new copies of all objects contained within it. This is a complete, independent clone.

The diagram below visualizes the difference for a nested list `L1 = [['a'], ['b']]`:

![Shallow vs. Deep Copy Behavior](../images/png/1.3-copy-behavior.png)

As the diagram shows, after a **shallow copy**, both the original (`L1`) and the copy (`L2`) contain references to the *same* underlying sub-lists (`@200` and `@300`). Modifying a sub-list through `L2` will also change it for `L1`. After a **deep copy**, the copy (`L3`) is entirely independent; it has its own new sub-lists (`@600` and `@700`).

In [ ]:
import copy
nested_list = [[1, 2], [3, 4]]

# Shallow copy
shallow_copy = copy.copy(nested_list)
shallow_copy[0].append(99) # Modify a nested list

print(f"Original after shallow copy modification: {nested_list}") # The original is also changed!

# Deep copy
nested_list_2 = [[1, 2], [3, 4]]
deep_copy = copy.deepcopy(nested_list_2)
deep_copy[0].append(99)

print(f"Original after deep copy modification:   {nested_list_2}") # The original is safe

#### 3.3 Garbage Collection: How Python Reclaims Memory

Python's memory management is largely automatic, handled by a process called **garbage collection**. The primary mechanism is **reference counting**. Every object in memory has a counter that tracks how many variables (names) are currently pointing to it. When this count drops to zero, the object is immediately deallocated, and its memory is freed.

You can inspect the reference count of an object (for debugging purposes) using `sys.getrefcount()`. Note that this function's result is always at least 1 higher than you'd expect, because the function call itself creates a temporary reference.

**Reference Cycles and the Cycle Detector:**
Reference counting alone cannot handle **reference cycles**. This occurs when a set of objects refer to each other, so their reference counts never drop to zero, even if they are no longer accessible from anywhere else in the program. For example, `a` points to `b`, and `b` points back to `a`.

To solve this, Python has a secondary garbage collection mechanism: a **cycle detector**. Periodically, this algorithm runs to find these isolated cycles of objects and cleans them up. This is why you generally don't have to worry about memory leaks in Python, although they are still possible (e.g., if a long-lived global dictionary keeps accumulating objects).

In [ ]:
import sys
import gc

# --- Reference Counting ---
a = []
print(f"Initial refcount for list 'a': {sys.getrefcount(a)}") # Starts at 2 (a and the function argument)
b = a
print(f"Refcount after 'b = a': {sys.getrefcount(a)}")
b = None
print(f"Refcount after 'b = None': {sys.getrefcount(a)}")
a = None # Refcount will drop to 0, object is deallocated

# --- Creating a Reference Cycle ---
class MyClass:
    def __init__(self, name):
        self.name = name
        print(f"Created {self.name}")
    def __del__(self):
        # This special method is called when an object is about to be destroyed
        print(f"Destroying {self.name}")

print("\n--- Demonstrating the Cycle Detector ---")
obj1 = MyClass('obj1')
obj2 = MyClass('obj2')
obj1.other = obj2 # obj1 points to obj2
obj2.other = obj1 # obj2 points back to obj1

del obj1
del obj2

# At this point, the objects are unreachable, but their refcounts are not zero.
# They are 'garbage', but not collected yet.
# We can manually run the cycle detector to see them get cleaned up.
print("Manually running garbage collector...")
gc.collect() # This will find and break the cycle


#### 3.4 Mutability and Function Arguments: A Common Pitfall

This leads to a classic Python gotcha: **never use a mutable type as a default argument in a function definition.** Default arguments are evaluated and created only *once*, at the moment the function is defined (i.e., when the `def` statement is executed), not each time the function is called. If that default argument is a mutable object (like a list or dictionary), it will be the *exact same object* shared across all subsequent calls to that function that don't explicitly provide a value for that argument.

This behavior is a direct consequence of the fact that functions are objects in Python, and their default arguments are stored as part of the function object's state.


In [ ]:
# The WRONG way: Using a mutable default argument
def add_to_list_buggy(item, my_list=[]):
    # We can inspect the default argument's memory ID
    my_list.append(item)
    return my_list

print("Calling buggy function:")
list1 = add_to_list_buggy(1)
print(f"Call 1 result: {list1}")
list2 = add_to_list_buggy(2)
print(f"Call 2 result: {list2} (Unexpected!)")

print("\n" + "-"*50 + "\n")

# The CORRECT idiom: Use `None` as a sentinel value
def add_to_list_correct(item, my_list=None):
    if my_list is None:
        my_list = []
    my_list.append(item)
    return my_list

print("Calling correct function:")
list3 = add_to_list_correct(1)
print(f"Correct Call 1 result: {list3}")
list4 = add_to_list_correct(2)
print(f"Correct Call 2 result: {list4}")

### 4. Operators

#### 4.1 Arithmetic and Comparison Operators
Python uses standard operators. It's crucial to understand **operator precedence** to avoid subtle bugs. When in doubt, always use parentheses `()` to make the order of operations explicit.

**Common Operator Precedence (from highest to lowest):**
1.  `**` (Exponentiation)
2.  `-` (Unary negation)
3.  `*`, `/`, `//`, `%` (Multiplication, Division, Floor Division, Modulo)
4.  `+`, `-` (Addition, Subtraction)
5.  `in`, `not in`, `is`, `is not`, `<`, `<=`, `>`, `>='`, `!=`, `==` (Comparisons)
6.  `not` (Logical NOT)
7.  `and` (Logical AND)
8.  `or` (Logical OR)

#### 4.2 Bitwise Operators

Bitwise operators act on integers as if they were strings of binary digits. They are rarely used in high-level economic modeling but are fundamental in computer science and can be useful for performance-critical algorithms or for working with flags.
- `&` (AND): Sets each bit to 1 only if both corresponding bits are 1.
- `|` (OR): Sets each bit to 1 if at least one of the corresponding bits is 1.
- `^` (XOR): Sets each bit to 1 only if the corresponding bits are different.
- `~` (NOT): Inverts all the bits.
- `<<` (Left Shift): Shifts bits to the left, effectively multiplying by 2 for each shift.
- `>>` (Right Shift): Shifts bits to the right, effectively dividing by 2 for each shift.

In [ ]:
a = 60  # 0011 1100
b = 13  # 0000 1101

print(f"a & b = {a & b}")   # 0000 1100 (12)
print(f"a | b = {a | b}")   # 0011 1101 (61)
print(f"a >> 2 = {a >> 2}") # 0000 1111 (15)

### 5. The Python Data Model: Understanding "Under the Hood"

One of Python's most powerful features is its consistent data model. The reason operators like `+`, `len()`, and `[]` work across different types is not a coincidence; it's because these types implement a standard set of special methods (often called "dunder" or "magic" methods) that Python calls under the hood. Understanding this concept, known as **operator overloading**, is key to creating intuitive and "Pythonic" classes.

For example, `a + b` is syntactic sugar for `a.__add__(b)`. The diagram below shows how common Python syntax is translated into method calls.

![A diagram showing the translation from Python syntax to data model methods.](../images/png/1.3-data-model-translation.png)

Here is a concrete example with a custom `Vector` class that implements several of these methods to behave like a numerical type.

In [ ]:
import math

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def __repr__(self): # Controls how the object is printed
        return f'Vector({self.x!r}, {self.y!r})' # !r calls repr() on the value

    def __abs__(self): # Controls the `abs()` built-in function
        return math.hypot(self.x, self.y)
    
    def __add__(self, other): # Controls the `+` operator
        return Vector(self.x + other.x, self.y + other.y)
    
    def __mul__(self, scalar): # Controls the `*` operator for scalar multiplication
        return Vector(self.x * scalar, self.y * scalar)
    
    def __len__(self): # Controls the `len()` built-in function
        return 2
    
    def __getitem__(self, index): # Controls sequence-like access with []
        if index == 0:
            return self.x
        elif index == 1:
            return self.y
        else:
            raise IndexError("Vector index out of range")
            
    def __eq__(self, other): # Controls the `==` operator
        return self.x == other.x and self.y == other.y

# --- Demonstrate the data model in action ---
v1 = Vector(2, 4)
v2 = Vector(2, 1)
v3 = Vector(2, 4)

print(f"v1 + v2 = {v1 + v2}")
print(f"abs(v1) = {abs(v1):.3f}")
print(f"v1 * 3 = {v1 * 3}")
print(f"len(v1) = {len(v1)}")
print(f"v1[0] (the x-component) = {v1[0]}")
print(f"Are v1 and v2 equal? {v1 == v2}")
print(f"Are v1 and v3 equal? {v1 == v3}")

### 6. Advanced Type Hinting

Python is a dynamically typed language, meaning you don't have to declare the type of a variable. While this allows for rapid development, it can make large codebases difficult to understand and debug. **Type hints**, introduced in Python 3.5, allow you to annotate your code with the *expected* types of variables and function arguments.

**Note:** Type hints are *not* enforced by the Python interpreter at runtime. Their purpose is to help developers and static analysis tools (like `mypy`) catch type-related errors before the code is even run. For runtime enforcement, libraries like `pydantic` are becoming increasingly popular.

The `typing` module provides a rich set of types for this purpose, including:
- `Optional[X]`: An optional value, equivalent to `Union[X, None]`. Used for arguments that can be `None`.
- `Union[X, Y]`: A value that can be one of several types.
- `Callable[[arg1_type, arg2_type], return_type]`: A function (or other callable).
- `Any`: A completely unrestricted type.

In [ ]:
from typing import List, Tuple, Optional, Callable

def get_portfolio_analyzer(method: str) -> Callable[[List[float], List[float]], float]:
    """Returns a function to analyze a portfolio."""
    if method == 'mean':
        def mean_return(weights, returns):
            return sum(w * r for w, r in zip(weights, returns))
        return mean_return
    elif method == 'variance':
        def variance(weights, returns):
            # Simplified variance calculation for demonstration
            mean = sum(w * r for w, r in zip(weights, returns))
            return sum(w * (r - mean)**2 for w, r in zip(weights, returns))
        return variance
    raise ValueError(f"Unknown method: {method}")

weights = [0.6, 0.4]
returns = [0.1, 0.05]

mean_analyzer = get_portfolio_analyzer('mean')
variance_analyzer = get_portfolio_analyzer('variance')

print(f"Mean return: {mean_analyzer(weights, returns):.4f}")
print(f"Variance:    {variance_analyzer(weights, returns):.4f}")

### 7. Exception Handling: Writing Robust Code
Real-world data and processes are messy. A robust program must anticipate and handle errors gracefully instead of crashing. Python's mechanism for this is the `try...except` block.
- **`try`**: The block of code that might raise an error.
- **`except [ErrorType]`**: If an error of the specified type occurs in the `try` block, the code in the `except` block is executed.
- **`else`**: An optional block that runs only if the `try` block completes *without* raising an error.
- **`finally`**: An optional block that *always* runs, regardless of whether an error occurred or not. This is essential for cleanup actions, like closing a file or a database connection.

A common use case in economics is processing data that may have missing values or incorrect types. A robust script must handle these cases without failing on the first problematic row.

In [ ]:
def calculate_debt_to_income(income: str, debt: str) -> Optional[float]:
    """Robustly calculates the debt-to-income ratio from raw string inputs."""
    try:
        income_f = float(income)
        debt_f = float(debt)
        if income_f <= 0:
            raise ValueError("Income must be positive.")
        ratio = debt_f / income_f
    except ValueError as e:
        # Catches errors from float() conversion or the explicit raise
        print(f"Error processing data: {e}. Returning None.")
        return None
    except ZeroDivisionError:
        print("Error: Income cannot be zero. Returning None.")
        return None
    else:
        print("Calculation successful.")
        return ratio
    finally:
        # This runs no matter what, useful for logging or cleanup.
        print("--- Finished processing attempt ---")

print("Case 1: Valid data")
calculate_debt_to_income("50000", "15000")

print("\nCase 2: Invalid type")
calculate_debt_to_income("fifty-thousand", "15000")

print("\nCase 3: Logical error (zero income)")
calculate_debt_to_income("0", "15000")

### 8. Exercises

#### Exercise 1: Catastrophic Cancellation
- Write a Python function `log_diff(x, y)` that naively computes `log(x) - log(y)`.
- Write a second function `log_diff_stable(x, y)` that computes the same value but using the mathematically equivalent and more stable formula `log(x/y)`.
- Test both functions with `x = 10.0000001` and `y = 10.0`. Which function gives a more accurate result and why?

#### Exercise 2: Mutability and Copying
- Create a list of lists: `nested_list = [[1, 2], [3, 4]]`.
- Create a *shallow copy* of this list: `shallow_copy = nested_list.copy()`.
- Modify an element of a nested list in the copy: `shallow_copy[0][0] = 99`.
- Print both `shallow_copy` and the original `nested_list`. Explain why the original list was also modified. How would you create a *deep copy* to avoid this? (Use the `copy` module).

#### Exercise 3: The Python Data Model
- The `Vector` class in this chapter is a good start. Now, extend it by implementing the `__sub__` method for vector subtraction and the `__rmul__` method to handle scalar multiplication when the scalar is on the left (e.g., `3 * v1`).

#### Exercise 4: Type Hinting
- Write a function `process_data(data: List[Optional[float]]) -> float:` that takes a list which may contain floats or `None` values.
- The function should filter out the `None` values and return the average of the remaining numbers. If the list is empty or contains only `None`, it should return 0.0.
- Demonstrate the function with a sample list like `[1.0, 2.5, None, 4.0, None]`.

--- 

### 9. Solutions to Exercises

In [ ]:
# Solution for Exercise 1
import math
def log_diff(x, y): return math.log(x) - math.log(y)
def log_diff_stable(x, y): return math.log(x/y)
x, y = 10.0000001, 10.0
print(f"Naive:  {log_diff(x,y):.17f}")
print(f"Stable: {log_diff_stable(x,y):.17f}")
# The stable version avoids subtracting two nearly-equal numbers.

# Solution for Exercise 2
import copy
nested_list = [[1, 2], [3, 4]]
shallow_copy = nested_list.copy()
shallow_copy[0][0] = 99
print(f"\nOriginal list was modified by shallow copy: {nested_list}")
deep_copy = copy.deepcopy(nested_list)
deep_copy[0][0] = 1 # Resetting the value to demo deepcopy
print(f"Original list is unaffected by deep copy: {nested_list}")

# Solution for Exercise 3
class VectorUpdated(Vector):
    def __sub__(self, other): return Vector(self.x - other.x, self.y - other.y)
    def __rmul__(self, scalar): return self * scalar # Multiplication is commutative
v1 = VectorUpdated(2, 4)
v2 = VectorUpdated(1, 1)
print(f"\nVector subtraction: v1 - v2 = {v1 - v2}")
print(f"Reflected multiplication: 3 * v1 = {3 * v1}")

# Solution for Exercise 4
from typing import List, Optional
def process_data(data: List[Optional[float]]) -> float:
    clean_data = [x for x in data if x is not None]
    if not clean_data: return 0.0
    return sum(clean_data) / len(clean_data)
sample_data = [1.0, 2.5, None, 4.0, None]
print(f"\nProcessed data average: {process_data(sample_data)}")